In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/datasets/nlp-getting-started.zip" -d "/content/datasets"

Archive:  /content/drive/MyDrive/Colab Notebooks/datasets/nlp-getting-started.zip
  inflating: /content/datasets/sample_submission.csv  
  inflating: /content/datasets/test.csv  
  inflating: /content/datasets/train.csv  


In [ ]:
df_train = pd.read_csv('/content/datasets/train.csv')
df_train.tail(5)

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [ ]:
df_test = pd.read_csv('/content/datasets/test.csv')
df_test.tail(5)

,id,keyword,location,text
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,10875,NaN,NaN,#CityofCalgary has activated its Municipal Eme...


In [ ]:
_, class_names = pd.factorize(df_train.target)
class_names.to_list()

[1, 0]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_train.text	, df_train.target, test_size=.1, random_state=28, stratify=df_train.target)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(6851,) (6851,)
(762,) (762,)


In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00


# BERT

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow as tf

def tokenize_text(text):
    tokens = tokenizer.encode_plus(text, add_special_tokens=True, max_length=256, padding='max_length', truncation=True)
    return tokens['input_ids'], tokens['attention_mask']

X_train_tokens = [tokenize_text(text) for text in X_train]
X_val_tokens = [tokenize_text(text) for text in X_val]

train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': [input_ids for input_ids, _ in X_train_tokens],
    'attention_mask': [attention_mask for _, attention_mask in X_train_tokens]
}, y_train))

val_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': [input_ids for input_ids, _ in X_val_tokens],
    'attention_mask': [attention_mask for _, attention_mask in X_val_tokens]
}, y_val))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy('sparse_categorical_accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(
    train_dataset.shuffle(100).batch(16),
    epochs=10,
    validation_data=val_dataset.batch(16)
)

Epoch 1/10
429/429 [==============================] - 170s 305ms/step - loss: 0.5918 - sparse_categorical_accuracy: 0.7081 - val_loss: 0.4814 - val_sparse_categorical_accuracy: 0.7913
Epoch 2/10
429/429 [==============================] - 128s 299ms/step - loss: 0.4607 - sparse_categorical_accuracy: 0.8024 - val_loss: 0.4063 - val_sparse_categorical_accuracy: 0.8281
Epoch 3/10
429/429 [==============================] - 129s 300ms/step - loss: 0.4142 - sparse_categorical_accuracy: 0.8254 - val_loss: 0.3873 - val_sparse_categorical_accuracy: 0.8373
Epoch 4/10
429/429 [==============================] - 129s 300ms/step - loss: 0.3914 - sparse_categorical_accuracy: 0.8375 - val_loss: 0.3803 - val_sparse_categorical_accuracy: 0.8386
Epoch 5/10
429/429 [==============================] - 129s 300ms/step - loss: 0.3729 - sparse_categorical_accuracy: 0.8483 - val_loss: 0.3776 - val_sparse_categorical_accuracy: 0.8438
Epoch 6/10
429/429 [==============================] - 129s 300ms/step - loss: 0.

In [ ]:
def tokenize_text(text):
    tokens = tokenizer.encode_plus(text, add_special_tokens=True, max_length=256, padding='max_length', truncation=True)
    return tokens['input_ids'], tokens['attention_mask']

X_test_tokens = [tokenize_text(text) for text in df_test['text']]

test_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': [input_ids for input_ids, _ in X_test_tokens],
    'attention_mask': [attention_mask for _, attention_mask in X_test_tokens]
}, np.zeros(len(X_test_tokens))))

predictions = model.predict(test_dataset.batch(16))
predicted_labels = np.argmax(predictions.logits, axis=1)
df_test['predicted_label'] = predicted_labels
df_test[['id', 'text', 'predicted_label']]

204/204 [==============================] - 23s 101ms/step


,id,text,predicted_label
0,0,Just happened a terrible car crash,0
1,2,"Heard about #earthquake is different cities, s...",1
2,3,"there is a forest fire at spot pond, geese are...",1
3,9,Apocalypse lighting. #Spokane #wildfires,1
4,11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0
3259,10865,Storm in RI worse than last hurricane. My city...,1
3260,10868,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [ ]:
submission = pd.read_csv('/content/datasets/sample_submission.csv')
submission.head(5)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
submission['target'] = df_test['predicted_label']
submission.sample(10)

,id,target
693,2253,1
1596,5395,1
1314,4331,0
383,1247,0
2849,9443,1
1558,5241,0
673,2187,0
2447,8178,1
825,2713,0
743,2424,0


In [ ]:
submission.to_csv('/content/BERT_lr1e-6.csv', index=False)

# RoBERTa

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = TFRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
import tensorflow as tf

def tokenize_text(text):
    tokens = tokenizer.encode_plus(text, add_special_tokens=True, max_length=384, padding='max_length', truncation=True)
    return tokens['input_ids'], tokens['attention_mask']

X_train_tokens = [tokenize_text(text) for text in X_train]
X_val_tokens = [tokenize_text(text) for text in X_val]

train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': [input_ids for input_ids, _ in X_train_tokens],
    'attention_mask': [attention_mask for _, attention_mask in X_train_tokens]
}, y_train))

val_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': [input_ids for input_ids, _ in X_val_tokens],
    'attention_mask': [attention_mask for _, attention_mask in X_val_tokens]
}, y_val))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-7)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy('sparse_categorical_accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(
    train_dataset.shuffle(100).batch(8),
    epochs=30,
    validation_data=val_dataset.batch(8)
)

Epoch 1/30
857/857 [==============================] - 265s 259ms/step - loss: 0.6892 - sparse_categorical_accuracy: 0.5704 - val_loss: 0.6804 - val_sparse_categorical_accuracy: 0.5709
Epoch 2/30
857/857 [==============================] - 216s 252ms/step - loss: 0.6813 - sparse_categorical_accuracy: 0.5745 - val_loss: 0.6755 - val_sparse_categorical_accuracy: 0.5709
Epoch 3/30
857/857 [==============================] - 216s 252ms/step - loss: 0.6757 - sparse_categorical_accuracy: 0.5777 - val_loss: 0.6683 - val_sparse_categorical_accuracy: 0.5709
Epoch 4/30
857/857 [==============================] - 216s 252ms/step - loss: 0.6679 - sparse_categorical_accuracy: 0.5860 - val_loss: 0.6580 - val_sparse_categorical_accuracy: 0.6719
Epoch 5/30
857/857 [==============================] - 216s 252ms/step - loss: 0.6564 - sparse_categorical_accuracy: 0.6389 - val_loss: 0.6415 - val_sparse_categorical_accuracy: 0.6955
Epoch 6/30
857/857 [==============================] - 215s 251ms/step - loss: 0.

In [ ]:
def tokenize_text(text):
    tokens = tokenizer.encode_plus(text, add_special_tokens=True, max_length=384, padding='max_length', truncation=True)
    return tokens['input_ids'], tokens['attention_mask']

X_test_tokens = [tokenize_text(text) for text in df_test['text']]

test_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': [input_ids for input_ids, _ in X_test_tokens],
    'attention_mask': [attention_mask for _, attention_mask in X_test_tokens]
}, np.zeros(len(X_test_tokens))))

predictions = model.predict(test_dataset.batch(16))
predicted_labels = np.argmax(predictions.logits, axis=1)
df_test['predicted_label'] = predicted_labels
df_test[['id', 'text', 'predicted_label']]

204/204 [==============================] - 34s 150ms/step


,id,text,predicted_label
0,0,Just happened a terrible car crash,1
1,2,"Heard about #earthquake is different cities, s...",1
2,3,"there is a forest fire at spot pond, geese are...",1
3,9,Apocalypse lighting. #Spokane #wildfires,1
4,11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1
3259,10865,Storm in RI worse than last hurricane. My city...,1
3260,10868,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [ ]:
submission = pd.read_csv('/content/datasets/sample_submission.csv')
submission.head(5)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
submission['target'] = df_test['predicted_label']
submission.sample(10)

,id,target
1286,4228,0
2607,8701,0
931,3069,1
341,1107,0
436,1413,0
2145,7182,0
502,1641,1
281,914,1
150,497,0
2987,9879,0


In [ ]:
submission.to_csv('/content/RoBERTa_lr1e-7_epochs20.csv', index=False)

# ALBERT

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AlbertForSequenceClassification, AdamW, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, return_tensors="pt")

train_labels = torch.tensor(list(y_train))
val_labels = torch.tensor(list(y_val))

In [ ]:
train_dataset = TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels)
val_dataset = TensorDataset(val_encodings["input_ids"], val_encodings["attention_mask"], val_labels)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-6, no_deprecation_warning=True)
num_epochs = 7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {average_train_loss:.4f}")

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            logits = outputs.logits
            _, predicted = torch.max(logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    average_val_loss = val_loss / len(val_loader)
    accuracy = correct / total
    print(f"Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy * 100:.2f}%")

OutOfMemoryError: ignored

In [ ]:
predictions = []

for index, row in df_test.iterrows():
    text = row['text']
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    predictions.append(predicted_class)

df_test['target'] = predictions
df_test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [ ]:
submission = pd.read_csv('/content/datasets/sample_submission.csv')
submission.head(5)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
submission['target'] = df_test['target']
submission.sample(10)

,id,target
1822,6158,0
873,2868,1
733,2383,1
1352,4461,1
2888,9558,0
510,1668,1
987,3261,1
2100,7041,0
332,1086,0
585,1903,0


In [ ]:
submission.to_csv('/content/ALBERT_lr1e-6_epochs7.csv', index=False)

# XLNet

In [ ]:
from transformers import AutoTokenizer, TFXLNetForSequenceClassification
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
model = TFXLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertF

In [ ]:
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=200, return_tensors='tf')
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=200, return_tensors='tf')

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('sparse_categorical_accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history = model.fit(
    x=train_encodings.input_ids,
    y=y_train,
    validation_data=(val_encodings.input_ids, y_val),
    batch_size=16,
    epochs=13,
    verbose=1
)

results = model.evaluate(val_encodings.input_ids, y_val, batch_size=16)
print("Loss:", results[0])
print("Accuracy:", results[1])

Epoch 1/13


429/429 [==============================] - 119s 183ms/step - loss: 0.6162 - sparse_categorical_accuracy: 0.6614 - val_loss: 0.5213 - val_sparse_categorical_accuracy: 0.7572
Epoch 2/13
429/429 [==============================] - 71s 167ms/step - loss: 0.4947 - sparse_categorical_accuracy: 0.7733 - val_loss: 0.4561 - val_sparse_categorical_accuracy: 0.7795
Epoch 3/13
429/429 [==============================] - 71s 166ms/step - loss: 0.4467 - sparse_categorical_accuracy: 0.7957 - val_loss: 0.4686 - val_sparse_categorical_accuracy: 0.7861
Epoch 4/13
429/429 [==============================] - 71s 165ms/step - loss: 0.4247 - sparse_categorical_accuracy: 0.8100 - val_loss: 0.4076 - val_sparse_categorical_accuracy: 0.8163
Epoch 5/13
429/429 [==============================] - 71s 165ms/step - loss: 0.4016 - sparse_categorical_accuracy: 0.8264 - val_loss: 0.4052 - val_sparse_categorical_accuracy: 0.8241
Epoch 6/13
429/429 [==============================] - 71s 166ms/step - loss: 0.3941 - sparse_ca

In [ ]:
test_encodings = tokenizer(df_test['text'].tolist(), truncation=True, padding=True, max_length=200, return_tensors='tf')
test_input_ids = test_encodings['input_ids']

predictions = model.predict(test_input_ids)
predicted_labels = [1 if pred[1] > 0.5 else 0 for pred in predictions]

102/102 [==============================] - 10s 80ms/step


In [ ]:
submission = pd.read_csv('/content/datasets/sample_submission.csv')
submission.head(5)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
submission['target'] = predicted_labels
submission.sample(10)

,id,target
1968,6640,1
2914,9647,1
780,2577,0
2113,7083,0
3210,10654,0
2086,7004,0
968,3214,0
2812,9352,1
1637,5516,0
2148,7200,1


In [ ]:
submission.to_csv('/content/ALBERT_lr1e-6_epochs10.csv', index=False)